In [2]:
import pandas as pd
import os
import sys
from Bio.SubsMat import MatrixInfo as matlist
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

def fasta2List(pathFasta):
    f = open(pathFasta, "r")
    title = []
    seq = []
    seq_temp = []
    for line in f:
        if line[0] == ">":
            seq.append(''.join(seq_temp).replace("\n", ""))
            title.append(line.replace("\n", ""))
            seq_temp = []
        else:
            seq_temp.append(line)
    seq.append(''.join(seq_temp).replace("\n", ""))
    seq.pop(0)
    dictionary = dict(zip(title, seq))
    return dictionary

error_list = pd.read_csv("../../data/mismatch-analysis/uniprot-error-mismatch/uniprot_new_errors_filt.txt", sep=" ", header=None)
    

In [127]:
# Flagging des mismatch conservés
counter = 0
for index, row in error_list.iloc[:,:].iterrows():

    conserved = 0

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]

    mismtaching_seq = protein_Mismatch[0][row[3]:row[4]]
    for i, j in mySeq.items():
        myAlign = pairwise2.align.localms(j, mismtaching_seq, 2, 0, -.5, -.1, one_alignment_only=True, score_only=True)
        #print(format_alignment(*myAlign[0]))
        if myAlign/len(mismtaching_seq) > 1.8:
            conserved +=1

    if conserved >= 4:
        print(str(index)+'\t'.join([str(x) for x in row]))
        counter +=1
print(counter)


2214/home/meyer/uniprot/Q03721.id.fasta.mafft	SEQ_ERROR3	H2N6G1	603	623	602	631
12382/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	G1QQR1	512	587	511	576
12386/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	F6YTR7	511	587	510	576
12387/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	A0A096MTY2	511	587	510	576
12388/home/meyer/uniprot/Q05C16.id.fasta.mafft	SEQ_ERROR3	G3QPK1	512	587	511	576
12450/home/meyer/uniprot/Q07283.id.fasta.mafft	SEQ_ERROR3	A0A0D9S5Q7	870	881	1349	1366
12452/home/meyer/uniprot/Q07283.id.fasta.mafft	SEQ_ERROR3	A0A0D9S5Q7	1064	1075	1596	1608
12528/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	A0A0D9RJ19	1043	1246	1029	1055
12529/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	G1REY1	1012	1215	1029	1055
12530/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	F6QZS4	1038	1241	1029	1055
12532/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	F7E1W0	1043	1116	1029	1053
12533/home/meyer/uniprot/Q08174.id.fasta.mafft	SEQ_ERROR3	A0A096NY96	103

In [3]:
# Flagging des erreurs d'alignement
counter = 0
error_counter = 0
for index, row in error_list.iloc[:,:].iterrows():

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]
    human_prot_Mismatch = [val for key, val in mySeq.items() if row[0][20:-16] in key]
    if human_prot_Mismatch == []:
        error_counter += 1
        pass
    else:
        mismtaching_seq = protein_Mismatch[0]
        human_seq = human_prot_Mismatch[0][row[5]:row[6]]

        myAlign = pairwise2.align.localms(mismtaching_seq, human_seq, 2, 0, -.5, -.1, one_alignment_only=True, score_only=True)
        if myAlign/len(human_seq) > 1.8:
            print(str(index)+'\t'.join([str(x) for x in row]))
            counter += 1

print(counter)


795/home/meyer/uniprot/A4FU49.id.fasta.mafft	SEQ_ERROR3	F7HEK4	622	685	506	517
796/home/meyer/uniprot/A4FU49.id.fasta.mafft	SEQ_ERROR3	A0A096N2Z0	506	568	506	517
1030/home/meyer/uniprot/A6NE01.id.fasta.mafft	SEQ_ERROR3	A0A096NRY4	1145	1157	1145	1203
1385/home/meyer/uniprot/A6NK75.id.fasta.mafft	SEQ_ERROR3	G7PX39	342	353	351	362
1532/home/meyer/uniprot/A6NMB1.id.fasta.mafft	SEQ_ERROR3	H0X9V5	417	428	388	399
1755/home/meyer/uniprot/A6QL64.id.fasta.mafft	SEQ_ERROR3	A0A096NRE9	241	262	743	757
1757/home/meyer/uniprot/A6QL64.id.fasta.mafft	SEQ_ERROR3	G3R883	438	451	931	944
1758/home/meyer/uniprot/A6QL64.id.fasta.mafft	SEQ_ERROR3	G3R883	562	585	1251	1274
1828/home/meyer/uniprot/A8K8V0.id.fasta.mafft	SEQ_ERROR3	H2NQP1	591	688	89	310
1829/home/meyer/uniprot/A8K8V0.id.fasta.mafft	SEQ_ERROR3	H2NQP1	739	757	363	381
2038/home/meyer/uniprot/A8MYU2.id.fasta.mafft	SEQ_ERROR3	G7PD63	102	116	101	118
2040/home/meyer/uniprot/A8MYU2.id.fasta.mafft	SEQ_ERROR3	F7HHD9	102	116	101	118
2151/home/meyer/uniprot/B

In [49]:
# Flagging des seq à repeats
counter = 0
error_counter = 0
for index, row in error_list.iloc[:,:].iterrows():

    mySeq = fasta2List("../../data/raw/uniprot-sequence/"+row[0][19:-6])
    protein_Mismatch = [val for key, val in mySeq.items() if row[2] in key]
    human_prot_Mismatch = [val for key, val in mySeq.items() if row[0][20:-16] in key]
    if human_prot_Mismatch == []:
        error_counter += 1
        pass
    else:
        mismtaching_seq = protein_Mismatch[0]
        human_seq = human_prot_Mismatch[0][row[5]:row[6]]

        begin_seq = human_prot_Mismatch[0][:row[5]+1]
        end_seq = human_prot_Mismatch[0][row[6]+2:]

        human_prot_Mismatch = begin_seq + end_seq
        myAlign = pairwise2.align.localms(human_prot_Mismatch, human_seq, 2, -1, -.5, -.1, one_alignment_only=True, score_only=True)
        if myAlign/len(human_seq) > 1.8:
            print(str(index)+'\t'.join([str(x) for x in row]))
            counter += 1

print(counter)

34/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	A0A0D9S6L1	443	466	404	430
35/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	A0A0D9S6L1	470	535	434	1459
36/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	A0A0D9S6L1	536	557	1460	1653
37/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	A0A0D9S6L1	559	615	1655	1710
38/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	A0A0D9S6L1	622	638	1717	3366
39/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	A0A0D9S6L1	640	683	3368	3750
42/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	G7NTQ3	102	115	447	1659
43/home/meyer/uniprot/A0A087WUL8.id.fasta.mafft	SEQ_ERROR3	G7NTQ3	198	214	3471	3749
609/home/meyer/uniprot/A1L4H1.id.fasta.mafft	SEQ_ERROR3	H0WN89	1168	1182	1238	1252
1000/home/meyer/uniprot/A6NDX5.id.fasta.mafft	SEQ_ERROR3	H0XML9	333	358	391	415
1002/home/meyer/uniprot/A6NDX5.id.fasta.mafft	SEQ_ERROR3	H0XML9	393	434	464	501
1614/home/meyer/uniprot/A6NN14.id.fasta.mafft	SEQ_ERROR3	H0WNG2	3

In [93]:
#Flagging des seq à N
myExonicMap = pd.read_csv("../../data/mismatch-analysis/uniprot-mismatch-colocalize/mismatch_exon_pos.tab", sep="\t", header=None)
exon_file = pd.read_csv("../../data/mismatch-analysis/uniprot-mismatch-colocalize/mismatch_exon_seq.tab", sep="\t", header=None)
intron_file = pd.read_csv("../../data/mismatch-analysis/uniprot-mismatch-colocalize/mismatch_intron_seq.tab", sep="\t", header=None)

counter = 0
for index, row in error_list.iloc[:,:].iterrows():
    my_CDS = []
    subset_exon_pos = myExonicMap.loc[(myExonicMap[0]==row[0]) & (myExonicMap[2]==row[2]) & \
                      (myExonicMap[3]== str(row[3])) & (myExonicMap[4]== str(row[4]))]
    subset_exon = exon_file.loc[exon_file[0]== row[2]]
    subset_intron = intron_file.loc[intron_file[0]== row[2]]
    try:
        for n in range(int(subset_exon_pos[5]), int(subset_exon_pos[6])+1):
            my_exon = subset_exon.loc[subset_exon[3] == n]
            my_intron = subset_intron.loc[subset_intron[3] == n]
            try:
                my_CDS.append(my_exon.iloc[0,6])
                my_CDS.append(my_intron.iloc[0,6])
            except:
                pass
        
        if "N" in ''.join(my_CDS):
            counter +=1
            print(str(index)+'\t'.join([str(x) for x in row]))
    except:
        pass

print(counter)

yer/uniprot/Q5W0A0.id.fasta.mafft	SEQ_ERROR3	H0XEI5	319	345	319	347
17540/home/meyer/uniprot/Q5W0B7.id.fasta.mafft	SEQ_ERROR3	H2N9W1	69	86	154	171
17541/home/meyer/uniprot/Q5W0B7.id.fasta.mafft	SEQ_ERROR3	H2N9W1	161	188	246	347
17543/home/meyer/uniprot/Q5W0Q7.id.fasta.mafft	SEQ_ERROR3	H0XDD2	517	621	526	647
17544/home/meyer/uniprot/Q5W0Q7.id.fasta.mafft	SEQ_ERROR3	H0XDD2	809	829	836	854
17545/home/meyer/uniprot/Q5W0Q7.id.fasta.mafft	SEQ_ERROR3	H0XDD2	836	852	861	875
17546/home/meyer/uniprot/Q5W0Q7.id.fasta.mafft	SEQ_ERROR3	H0XDD2	973	985	995	1007
17573/home/meyer/uniprot/Q5XXA6.id.fasta.mafft	SEQ_ERROR3	G1RHL3	570	592	592	626
17579/home/meyer/uniprot/Q63HM1.id.fasta.mafft	SEQ_ERROR3	H0X771	49	82	48	79
17580/home/meyer/uniprot/Q63HM1.id.fasta.mafft	SEQ_ERROR3	G1QGS6	6	18	5	17
17589/home/meyer/uniprot/Q63HN8.id.fasta.mafft	SEQ_ERROR3	H0WTB7	1629	1679	3563	3617
17596/home/meyer/uniprot/Q63HN8.id.fasta.mafft	SEQ_ERROR3	H0WTB7	2709	2720	4649	4660
17599/home/meyer/uniprot/Q63HN8.id.fasta.maf

In [ ]:
import pandas as pd 
intron_file = pd.read_csv("../raw/uniprot-exon-map/Intron_map.tab", sep="\t", header=None)
exon_file = pd.read_csv("../raw/uniprot-exon-map/Exon_map.tab", sep="\t", header=None)
ID_file = pd.read_csv("../raw/uniprot-exon-map/transcript_ensembl.tab", sep = "\t")
Error_file = pd.read_csv("../raw/uniprot-exon-map/uniprot_new_errors_filt.txt", sep=" ", header=None)
genomicFile = fasta2List("../raw/uniprot-exon-map/genomics_new.fa")
my_CDS = fasta2List("../raw/uniprot-exon-map/cds_new.fa")